<a href="https://colab.research.google.com/github/LinYenShou/114-1/blob/main/HW2_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install -q google-generativeai

In [24]:
import gradio as gr
import pandas as pd
from google.colab import auth
from google.auth import default

# -*- coding: utf-8 -*-
import gspread
from datetime import datetime
import google.generativeai as genai
import os
import json

In [25]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('gemini')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [26]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1-7xUMjl7-QQCpixs_bQT16qYVcjkj94VAYh7w1xdlZM/edit?gid=0#gid=0"
WORKSHEET_NAME = "工作表1"

REQUIRED_COLUMNS = ["日期", "科目", "作業成績","平均","標準差"]

_auth_done = False
_gc = None
_ws = None

In [ ]:
from datetime import datetime
import math # <--- 1. 導入 math 模組用於計算標準差

# --- 主要功能區塊 ---
def get_user_grades():
    """
    透過終端機輸入學生成績，直到使用者輸入 'q' 結束。
    """
    print("--- 準備輸入成績。輸入 'q' 來停止。---")
    grades = []
    while True:
        subject = input("請輸入科目（或輸入 'q' 停止）：")
        if subject.lower() == 'q':
            break

        grade_input = input(f"請輸入 {subject} 的成績：")
        try:
            grade = int(grade_input)
        except ValueError:
            print("❌ 成績必須是數字。請重新輸入。")
            continue

        today = datetime.now().strftime('%Y-%m-%d')
        # 儲存的成績是一個列表 [日期, 科目, 分數]
        grades.append([today, subject, grade])
        print(f"✅ 已記錄：日期: {today}, 科目: {subject}, 成績: {grade}\n")

    return grades

def analyze_grades(grades_data):
    """
    計算並輸出成績的平均值和標準差。
    """
    if not grades_data:
        return # 如果沒有成績，就停止計算

    # 2. 提取所有分數
    scores = [record[2] for record in grades_data]
    n = len(scores)

    # 3. 計算平均值 (Average)
    average = sum(scores) / n

    # 4. 計算標準差 (Standard Deviation)
    # 步驟 a: 計算每個分數與平均值的差的平方和
    variance_sum = sum([(score - average) ** 2 for score in scores])

    # 步驟 b: 計算變異數
    variance = variance_sum / n

    # 步驟 c: 開根號得到標準差
    std_dev = math.sqrt(variance)

    # 5. 輸出結果
    print("\n--- 成績統計分析 ---")
    print(f"總共輸入科目數： {n} 門")
    print(f"總分： {sum(scores)}")
    print(f"平均成績 (Average)： {average:.2f}")  # 保留兩位小數
    print(f"標準差 (Standard Deviation)： {std_dev:.2f}") # 保留兩位小數
    print("--------------------")

# --- 程式執行入口 ---
if __name__ == "__main__":
    all_grades = get_user_grades()

    # 在輸入完成後呼叫分析函式
    analyze_grades(all_grades)

    # 輸出所有原始記錄
    print("\n--- 原始輸入資料 ---")
    for record in all_grades:
        print(f"日期: {record[0]}, 科目: {record[1]}, 成績: {record[2]}")
    print("--------------------")

--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：a
請輸入 a 的成績：10
✅ 已記錄：日期: 2025-10-03, 科目: a, 成績: 10

請輸入科目（或輸入 'q' 停止）：b
請輸入 b 的成績：20
✅ 已記錄：日期: 2025-10-03, 科目: b, 成績: 20

請輸入科目（或輸入 'q' 停止）：q

--- 成績統計分析 ---
總共輸入科目數： 2 門
總分： 30
平均成績 (Average)： 15.00
標準差 (Standard Deviation)： 5.00
--------------------

--- 原始輸入資料 ---
日期: 2025-10-03, 科目: a, 成績: 10
日期: 2025-10-03, 科目: b, 成績: 20
--------------------


In [ ]:
# 請注意：假設您已在程式碼開頭導入了 math, genai 等模組

def calculate_stats(grades_data):
    """
    計算成績的平均值和標準差。
    返回一個包含 (average, std_dev) 的元組。
    """
    if not grades_data:
        return 0, 0

    # 確保成績是數字，因為 record[2] 來自外部輸入，可能需要轉換
    # 這裡假設 grades_data 的結構是 (日期, 科目名稱, 分數)
    scores = [record[2] for record in grades_data]

    # 如果分數是以字串形式存儲，請確保轉換為浮點數或整數
    # 範例：scores = [float(record[2]) for record in grades_data]

    n = len(scores)

    average = sum(scores) / n

    variance_sum = sum([(score - average) ** 2 for score in scores])
    variance = variance_sum / n
    std_dev = math.sqrt(variance)

    return round(average, 2), round(std_dev, 2)


# --- 核心 AI 呼叫函式 (已修改) ---
def get_ai_summary_and_review(grades):
    """
    呼叫 Gemini 模型，整合所有成績與統計數據，並要求生成總結評語、表現分析和學習建議。
    """
    # 1. 確保你已經配置了 Gemini API Key (這裡我們假設 client 已在 main 函式中初始化)
    # 這裡為了簡化，直接使用 GenerativeModel 語法，但推薦使用 client.models.get()
    model = genai.GenerativeModel('gemini-2.5-flash')

    # 2. 關鍵步驟：計算統計數據，並將結果存入 avg 和 std_dev
    avg, std_dev = calculate_stats(grades)

    # 3. 準備給 AI 的提示：將統計數據插入
    prompt_text = "你是一位資深教育顧問。請根據提供的學生成績和統計數據，生成一份詳細的報告。這份報告必須包含以下三個部分：\n\n"
    prompt_text += "1. **整體表現評語 (Review)：** 針對平均、科目數量和標準差，給予簡潔有力的總結評語。\n"
    prompt_text += "2. **表現分析與優勢/弱項 (Analysis)：** 分析成績的集中度（看標準差）和科目分數的分佈，指出可能的優勢領域和需要加強的弱項。\n"
    prompt_text += "3. **常見學習迷思與建議 (Advice)：** 提供針對此成績模式的常見學習迷思整理和實用建議。\n\n"

    prompt_text += "【統計數據】\n"

    # <<< 關鍵修改：使用 f-string 導入 avg 和 std_dev >>>
    prompt_text += f"總平均成績：{avg}\n"
    prompt_text += f"成績標準差：{std_dev}\n"
    prompt_text += "【統計數據】\n"
    prompt_text += f"總平均成績：{avg}\n"
    prompt_text += f"成績標準差：{std_dev}\n"
    prompt_text += f"總科目數：{len(grades)}\n\n"
    prompt_text += f"總科目數：{len(grades)}\n\n"
    prompt_text += "【原始成績】\n"
    for record in grades:
        date, subject, grade = record
        prompt_text += f"日期：{date}, 科目：{subject}, 成績：{grade}\n"

    print("\n--- 正在呼叫 AI 模型生成報告... ---")
    try:
        response = model.generate_content(prompt_text)
        report = response.text
        return report
    except Exception as e:
        print(f"呼叫 AI 時發生錯誤：{e}")
        return "AI 報告生成失敗。"

In [ ]:
new_grades = get_user_grades()

--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：a
請輸入 a 的成績：10
✅ 已記錄：日期: 2025-10-03, 科目: a, 成績: 10

請輸入科目（或輸入 'q' 停止）：b
請輸入 b 的成績：20
✅ 已記錄：日期: 2025-10-03, 科目: b, 成績: 20

請輸入科目（或輸入 'q' 停止）：q


In [ ]:
new_grades

[['2025-10-03', 'a', 10], ['2025-10-03', 'b', 20]]

In [ ]:
get_ai_summary_and_review(new_grades)


--- 正在呼叫 AI 模型生成報告... ---


'## 學生學業表現詳細報告\n\n**學生姓名：** [未提供，可自行填入]\n**報告日期：** 2023年10月27日\n**顧問：** 資深教育顧問\n\n---\n\n### 一、 整體表現評語 (Overall Review)\n\n本報告基於學生目前極其有限的兩科目成績數據進行分析。\n\n*   **總平均成績 15.0：** 在未明確總分基準的情況下，若假定滿分為20分，則此平均值屬於中等偏上的表現。若滿分為100分，則該平均值表現不佳。鑒於單科成績為10和20，本報告將暫時以「滿分20分」的參考標準進行解讀。在此前提下，平均15.0分顯示學生具備一定的學習潛力。\n*   **總科目數 2：** 僅有兩科目成績，這是一個非常小且不具代表性的樣本。因此，任何基於此數據得出的結論都應被視為初步觀察，無法全面反映學生的整體學習能力或學術潛力。\n*   **成績標準差 5.0：** 相對於平均值15.0分而言，5.0的標準差在僅有兩個科目數據的情況下，反映出**科目之間成績的高度分散**。這意味著學生的學習表現存在顯著的科目不平衡性。\n\n**總結而言，目前的數據描繪了一位在少數科目中表現極度不均的學生，擁有一項突出優勢但同時存在一項明顯弱項的初步學習模式。**\n\n---\n\n### 二、 表現分析與優勢/弱項 (Performance Analysis & Strengths/Weaknesses)\n\n#### 表現分析：\n\n1.  **成績集中度與分佈：**\n    *   標準差5.0清晰地揭示了學生兩科目成績的**兩極分化**。具體而言，科目a的成績為10，恰好比平均值低一個標準差；科目b的成績為20，恰好比平均值高一個標準差。這種分佈顯示學生的學習表現並非穩定一致，而是在不同學科領域展現出截然不同的成果。\n    *   這暗示學生可能在不同科目之間存在明顯的興趣差異、基礎知識掌握程度不同，或者學習方法適用性的問題。\n    *   從宏觀角度看，這種高度分散的成績分布在早期階段既是挑戰，也提供了明確的改進方向。\n\n2.  **科目分數分佈：**\n    *   **科目b (成績：20)：** 若以滿分20分計，學生在科目b中取得了**滿分**。這是一個非常令人振奮的成績，表明學生在該科目中可能展現出：\n   

In [28]:
def main():
    """
    主程式流程：輸入成績 -> 獲取 AI 摘要與統計數據 -> 寫入 Google Sheet。
    """
    try:
        # 1. Google Sheet 身份驗證 (此區塊不變)
        auth.authenticate_user()

        creds, _ = default()
        gc = gspread.authorize(creds)

        # 假設 SHEET_URL 和 WORKSHEET_NAME 已在其他地方定義
        sh = gc.open_by_url(SHEET_URL)
        ws = sh.worksheet(WORKSHEET_NAME)

        print("--- Google Sheet 連線成功。---")

        # 2. 獲取使用者輸入的成績
        new_grades = get_user_grades()

        if not new_grades:
            print("沒有輸入任何成績，程式結束。")
            return

        # 3. 將新成績寫入 Google Sheet
        ws.append_rows(new_grades)
        print("\n--- 成績已成功寫入 Google Sheet。---")

        # 4. 準備 AI 呼叫所需的數據

        # 關鍵步驟 A: 計算統計數據
        # 確保 calculate_stats 函式在其他地方有定義
        avg, std_dev = calculate_stats(new_grades)

        # 關鍵步驟 B: 創建包含統計數據的新列表給 AI
        grades_for_ai = new_grades.copy()
        current_date = datetime.now().strftime('%Y-%m-%d')

        # 將平均值和標準差作為特殊的「記錄」加入，這樣 AI 就能在原始數據中看到它們
        grades_for_ai.append([current_date, "---總平均---", avg])
        grades_for_ai.append([current_date, "---標準差---", std_dev])

        # 關鍵步驟 C: 呼叫 AI 函式，並傳入包含統計數據的列表
        # 💡 注意：我們需要修改 get_ai_summary_and_review 讓它接受並分析這個列表
        summary = get_ai_summary_and_review(grades_for_ai)

        # 5. 尋找並寫入 AI 摘要 (邏輯調整，讓 AI 摘要與統計數據共存)

        # 尋找第一行空白列 (將用於寫入 AI 摘要的標題和內容)
        next_row = len(ws.col_values(1)) + 1

        # 由於 AI 摘要可能很長，我們將統計數據寫在 AI 摘要之前的一行或兩行，方便查看

        # 寫入總平均 (在 AI 摘要標題前兩行)
        ws.update_cell(next_row, 1, current_date)
        ws.update_cell(next_row, 2, '總平均')
        ws.update_cell(next_row, 3, avg)

        # 寫入標準差 (在 AI 摘要標題前一行)
        ws.update_cell(next_row + 1, 1, current_date)
        ws.update_cell(next_row + 1, 2, '標準差')
        ws.update_cell(next_row + 1, 3, std_dev)

        # AI 摘要標題從 next_row + 2 開始
        ws.update_cell(next_row + 2, 1, current_date)
        ws.update_cell(next_row + 2, 2, 'AI 摘要')

        # 為了避免單元格內容過長，將摘要內容分成多行來寫入
        summary_lines = summary.split('\n')
        for i, line in enumerate(summary_lines):
            # 從第三欄 (C) 開始寫入內容
            ws.update_cell(next_row + 2 + i, 3, line)

        print("\n--- 統計數據和 AI 摘要已成功寫入 Google Sheet。---")
        print("以下是 AI 生成的摘要內容：")
        print("-" * 50)
        print(summary)
        print("-" * 50)

    except gspread.exceptions.APIError as e:
        # 修正 IndentationError：確保 except 區塊有縮排內容
        print(f"Google Sheets API 錯誤：{e.response.text}")
        print("請確認：")
        print("1. 您的服務帳戶金鑰檔案正確且未過期。")
        print("2. 您已將服務帳戶的 Email 地址（在 JSON 檔案中）分享給 Google Sheet，並給予編輯權限。")
    except Exception as e:
        # 修正 IndentationError：確保 except 區塊有縮排內容
        print(f"發生未預期的錯誤：{e}")

if __name__ == "__main__":
    main()

--- Google Sheet 連線成功。---
--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：a
請輸入 a 的成績：10
✅ 已記錄：日期: 2025-10-03, 科目: a, 成績: 10

請輸入科目（或輸入 'q' 停止）：b
請輸入 b 的成績：20
✅ 已記錄：日期: 2025-10-03, 科目: b, 成績: 20

請輸入科目（或輸入 'q' 停止）：q

--- 成績已成功寫入 Google Sheet。---

--- 正在呼叫 AI 模型生成報告... ---

--- 統計數據和 AI 摘要已成功寫入 Google Sheet。---
以下是 AI 生成的摘要內容：
--------------------------------------------------
## 學生學業表現評估報告

**日期：** 2023年10月26日
**致：** 家長/學生
**發件人：** 資深教育顧問

---

### **1. 整體表現評語 (Review)**

該生目前的學業表現呈現出「**中等偏低，但高度不穩定**」的特點。總平均成績為12.5分（若假設滿分為20分，則約為中等；若滿分為100分，則偏低）。在總共4個科目中，成績標準差高達5.59，這是一個非常顯著的離散度，遠超過平均分數的約45%，強烈暗示學生的學業表現極不均衡，在不同科目之間存在巨大落差。這表明學生可能在某些領域擁有卓越潛力，但在其他領域則面臨嚴峻挑戰。

### **2. 表現分析與優勢/弱項 (Analysis)**

**成績集中度與分佈：**
高標準差（5.59）是這份成績數據中最值得關注的指標。它明確指出學生的分數並非穩定地集中在平均值附近。這代表學生的學習成果並非是穩定的「普通」水平，而是由極高分與極低分共同拉扯而成的平均值。

雖然提供的原始成績只顯示了兩個科目 (a: 10分, b: 20分) 作為範例，它們卻完美地驗證了整體數據所呈現的高度不平衡。科目「b」獲得20分，這是一個非常優異的成績（若以20為滿分），顯示學生在該科目或相關領域具備很強的理解力、掌握度與學習能力。相比之下，科目「a」的10分則相對較低，表明該領域存在顯著的學習障礙或知識空白。這兩極化的分數分佈導致了整體平均值被拉低，同時也使得標準差